In [1]:
import xarray as xr
import xesmf as xe
import matplotlib.pyplot as plt
import glob
import os
import numpy as np
import datetime

/N/u/gongg/Quartz/anaconda3/envs/xesmf/lib/python3.10/site-packages/esmpy/interface/loadESMF.py:94: VersionWarning: ESMF installation version 8.8.0, ESMPy version 8.8.0b0
  warnings.warn("ESMF installation version {}, ESMPy version {}".format(


In [2]:
pattern_june = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/PREC_ACC_NC.wrf2d_d01_????-06-??_*.nc'
pattern_july = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/PREC_ACC_NC.wrf2d_d01_????-07-??_*.nc'
pattern_aug  = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/PREC_ACC_NC.wrf2d_d01_????-08-??_*.nc'
pattern_spet  = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/PREC_ACC_NC.wrf2d_d01_????-09-??_*.nc'


files_june = glob.glob(pattern_june)
files_july = glob.glob(pattern_july)
files_aug  = glob.glob(pattern_aug)
files_spet  = glob.glob(pattern_spet)

files_jja = sorted(files_june + files_july + files_aug + files_spet)

ds_ref1 = xr.open_mfdataset("ref_04deg.nc")
ds_ref2 = xr.open_mfdataset("ref_.1deg.nc")

test_ds_wrf1 = xr.open_dataset('/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/PREC_ACC_NC.wrf2d_d01_1980-06-01_00:00:00.nc')
test_ds_wrf2 = xr.open_dataset('/N/project/Zli_lab/gongg/CONUS404_data/regrid_bl/19800601.nc')


regridder1 = xe.Regridder(test_ds_wrf1, ds_ref1, method="bilinear", periodic=False)
regridder2 = xe.Regridder(test_ds_wrf2, ds_ref2, method="conservative", periodic=False)

### regrid to bilinear to conservative 0.1deg do not rerun!

In [10]:
chunk_size = 24
for i in range(0, 100, chunk_size): # 100 should be len(files_jja)
    chunk = files_jja[i : i + chunk_size]
    
    ds_wrf = xr.open_mfdataset(chunk, combine='by_coords')

    ds_wrf_regrid = regridder2(regridder1(ds_wrf))
    
    basename = os.path.basename(chunk[0])               
    date_str = basename.split("_")[4]                  
    yyyymmdd = date_str.replace("-", "")                

    out_file = f'/N/project/Zli_lab/gongg/CONUS404_data/regrid_cons/{yyyymmdd}.nc'

    ds_cleaned = ds_wrf_regrid.drop_vars('XTIME')
    ds_cleaned = ds_cleaned.rename({'Time': 'time'})
    
    ds_cleaned.to_netcdf(out_file)


2025-03-26 00:18:21.772103


### UTC to LST

In [2]:
pattern_UTC  = '/N/project/Zli_lab/gongg/CONUS404_data/regrid_bl/*.nc'
files_UTC = sorted(glob.glob(pattern_UTC))

lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
utc_offsets = [-8, -7, -6, -5]

for file_path in files_UTC[:1000]:

    base_name = os.path.basename(file_path)  # e.g. '19800602.nc'
    date_part = base_name.replace('.nc', '') # e.g. '19800602'
    year      = date_part[0:4]               # '1980'
    month_str = date_part[4:6]               # '06'
    day_str   = date_part[6:8]               # '02'
    

    ds_sss = xr.open_dataset(file_path)
    original_times = ds_sss.Time.values  # 假设有 24 小时或更多
    
    for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):

        mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
        ds_lon_subset = ds_sss.where(mask, drop=True)
        

        if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:

            adjusted_times = original_times + np.timedelta64(offset, 'h')
            ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
            lat_min = ds_lon_subset.lat.min().values
            lat_max = ds_lon_subset.lat.max().values
            lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
            for i in range(len(lat_splits) - 1):
                lat_min_split = lat_splits[i]
                lat_max_split = lat_splits[i + 1]
                lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                ds_lat_subset = ds_lat_subset.drop_vars("Time")
                ds_lat_subset = ds_lat_subset.drop_vars("XTIME")
                output_folder = f'/N/project/Zli_lab/gongg/CONUS404_data/regrid_bl_LST/U{offset}{i}/'
                os.makedirs(output_folder, exist_ok=True)
                
                output_file = f"{year}-{month_str}-{day_str}.nc"
                output_path = os.path.join(output_folder, output_file)
                
                ds_lat_subset.to_netcdf(output_path)


KeyboardInterrupt: 